In [ ]:
algorithm: P2P_Mid_Market_Rate_Billing
description: >
  Mid-Market Rate (MMR) pricing for a community microgrid.
  Internal prices are based on the mid-point between grid buy/sell prices
  and adjusted each time slot according to whether PV generation is
  equal to, higher than, or lower than community demand. 

inputs:
  H:        # integer, number of time slots in billing horizon (e.g., 96 for 15-min slots)
  dt:       # float, duration of each slot in hours
  N:        # integer, number of prosumers (households)
  lambda_buy_grid:   # c_BFG, grid buying price (retail)
  lambda_sell_grid:  # c_STG, grid selling price (feed-in tariff)
  L:        # matrix [N x H], L[i][h] = demand of prosumer i at slot h (kW)
  PV:       # matrix [N x H], PV[i][h] = PV generation of prosumer i at slot h (kW)

outputs:
  C:        # array [N], C[i] = final bill of prosumer i over horizon (monetary)
  cp2p:     # float, static mid-market reference price
  c_im:     # array [H], c_im[h] = actual internal buying price at slot h
  c_ex:     # array [H], c_ex[h] = actual internal selling price at slot h
  diagnostics:
    L_total:  # array [H], community total demand per slot
    PV_total: # array [H], community total PV per slot

constants_and_precomputation:
  # Mid-market reference price (eq. 15) :contentReference[oaicite:1]{index=1}
  cp2p: (lambda_buy_grid + lambda_sell_grid) / 2.0

  # Initialize arrays
  for h in 0..H-1:
    c_im[h]: 0.0
    c_ex[h]: 0.0
    L_total[h]: 0.0
    PV_total[h]: 0.0

  for i in 0..N-1:
    C[i]: 0.0

steps:

  - step: "Compute per-slot household overlaps and residuals"
    details:
      - for each time slot h in 0..H-1:
          # community aggregates
          L_total[h] = sum over i of L[i][h]
          PV_total[h] = sum over i of PV[i][h]

          # per-household instantaneous overlap and residuals (eqs. 1–3) :contentReference[oaicite:2]{index=2}
          for each prosumer i in 0..N-1:
            M_ih = min(L[i][h], PV[i][h])
            P_im_ih = L[i][h] - M_ih         # import residual
            P_ex_ih = PV[i][h] - M_ih         # export surplus

            # store P_im_ih, P_ex_ih for later billing
            Pim[i][h] = P_im_ih
            Pex[i][h] = P_ex_ih

  - step: "Compute time-varying internal prices with MMR"
    details:
      - for each time slot h in 0..H-1:
          # Case 1: PV generation == demand (within tolerance)
          if abs(PV_total[h] - L_total[h]) <= epsilon:
            c_im[h] = cp2p
            c_ex[h] = cp2p

          # Case 2: PV generation > demand (surplus PV) – eq. (16)b & (17)b
          # Buying price stays at cp2p; selling price is mix of cp2p and grid feed-in. :contentReference[oaicite:3]{index=3}
          else if PV_total[h] > L_total[h]:
            c_im[h] = cp2p
            c_ex[h] =
              ( L_total[h] * cp2p
                + (PV_total[h] - L_total[h]) * lambda_sell_grid
              ) / PV_total[h]

          # Case 3: PV generation < demand (shortage) – eq. (16)c & (17)c
          # Selling price is cp2p; buying price is mix of cp2p and grid retail. :contentReference[oaicite:4]{index=4}
          else:  # PV_total[h] < L_total[h]
            c_im[h] =
              ( PV_total[h] * cp2p
                + (L_total[h] - PV_total[h]) * lambda_buy_grid
              ) / L_total[h]
            c_ex[h] = cp2p

  - step: "Compute individual prosumer bills"
    details:
      - for each prosumer i in 0..N-1:
          energy_import_i = 0.0
          energy_export_i = 0.0

          # accumulate energy and cost with time-varying prices (eq. 18) :contentReference[oaicite:5]{index=5}
          for each time slot h in 0..H-1:
            energy_import_i += Pim[i][h] * dt
            energy_export_i += Pex[i][h] * dt

            C[i] += Pim[i][h] * dt * c_im[h]
            C[i] -= Pex[i][h] * dt * c_ex[h]

          # optional: store energy stats per prosumer
          E_import[i] = energy_import_i
          E_export[i] = energy_export_i

  - step: "Optional diagnostics and comparison"
    details:
      - compute community totals:
          E_im_total = sum over i of E_import[i]
          E_ex_total = sum over i of E_export[i]
      - conventional_bill =
          E_im_total * lambda_buy_grid - E_ex_total * lambda_sell_grid
      - mmr_total_bill = sum over i of C[i]
      - percent_saving =
          100 * (conventional_bill - mmr_total_bill) / conventional_bill

notes:
  - epsilon is a small tolerance (e.g., 1e-6) to decide “PV == demand”
    to avoid floating-point issues.
